In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
import datetime
import math
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df
num_partitions = 1000 #number of partitions to split dataframe
num_cores = 48 #number of cores on your machine



df_app2cat = pd.read_csv('data/App2Category.txt', sep="\t", header = None)
df_app2cat.columns = ["app_id", "cat_id"]
df_app2cat.head()

,app_id,cat_id
0,1,4
1,2,4
2,3,4
3,4,13
4,5,4


In [2]:
df_cat = pd.read_csv('data/Categorys.txt', sep="\t", header = None)
df_cat.columns = ["cat_id", "cat"]
df_cat

,cat_id,cat
0,0,'Utilities'
1,1,'Games'
2,2,'Entertainment'
3,3,'News'
4,4,"'Social\_Networking'.'wechat','linkedin','weibo'"
5,5,'Shopping'
6,6,'Finance'
7,7,'Business'
8,8,'Travel'
9,9,"'Lifestyle ','meituan',''?'"


In [3]:
df_usage = pd.read_csv('data/App_usage_trace.txt', sep=" ", header = None)
df_usage.columns = ["user_id", "timestamp","a","app_id",'bytes']
df_usage.head()

,user_id,timestamp,a,app_id,bytes
0,0,20160420081319,8194,361,1920
1,0,20160420081320,8194,361,1764
2,0,20160420081322,8194,361,4434
3,0,20160420081330,8194,361,703
4,0,20160420081331,8194,361,1682


In [4]:
# df_usage2 = df_usage.truncate(before = 20000 , after = 40000)
df_usage2 = df_usage
# df_usage2

In [5]:
cat_edu = [6,7,8,9,10,11,12,13,14,16,17,19]
cat_game = [0,1,2,5,15,18]
cat_social = [i for i in range(19) if i not in cat_edu and i not in cat_game]
print(cat_social)

[3, 4]


In [6]:
df_app2cat["cat"] = df_app2cat["cat_id"].apply(lambda x: "edu" if x in cat_edu else ("game" if x in cat_game else "social" ))
game = 0
edu = 0
social = 0
for i in df_app2cat["cat"]:
    if i == "edu": edu += 1
    elif i == "game": game += 1
    else: social += 1
print(edu, game , social)
df_app2cat.head()

402 1316 282


,app_id,cat_id,cat
0,1,4,social
1,2,4,social
2,3,4,social
3,4,13,edu
4,5,4,social


In [7]:
def add_cat(data):
    data["cat"] = data["app_id"].apply(lambda x: df_app2cat.loc[df_app2cat['app_id'] == x].values[0,2])
    return data
df_usage3 = parallelize_dataframe(df_usage2, add_cat)

df_usage3.head()

,user_id,timestamp,a,app_id,bytes,cat
0,0,20160420081319,8194,361,1920,game
1,0,20160420081320,8194,361,1764,game
2,0,20160420081322,8194,361,4434,game
3,0,20160420081330,8194,361,703,game
4,0,20160420081331,8194,361,1682,game


In [8]:
data = df_usage3[["user_id","timestamp","cat"]]
data.loc[:,"timestamp"] = pd.to_datetime(data["timestamp"],format="%Y%m%d%H%M%S")
data.head()

/home/student/.local/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,user_id,timestamp,cat
0,0,2016-04-20 08:13:19,game
1,0,2016-04-20 08:13:20,game
2,0,2016-04-20 08:13:22,game
3,0,2016-04-20 08:13:30,game
4,0,2016-04-20 08:13:31,game


In [9]:
data.loc[:,"date"] = [d.date() for d in data['timestamp']]
data.loc[:,"time"] = [d.time() for d in data['timestamp']]

data = data.loc[:,["user_id","date","time","cat"]]
data.head()

/home/student/.local/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


,user_id,date,time,cat
0,0,2016-04-20,08:13:19,game
1,0,2016-04-20,08:13:20,game
2,0,2016-04-20,08:13:22,game
3,0,2016-04-20,08:13:30,game
4,0,2016-04-20,08:13:31,game


In [10]:
df_out = pd.DataFrame(columns = ["Sc_Social","Ev_Social","Sl_Social","Sc_Edu","Ev_Edu","Sl_Edu","Sc_Game","Ev_Game","Sl_Game","Day1_Hs","Day1_D","Day2_Hs","Day2_D","Day3_Hs","Day3_D"]
                     ,index=[0])

df1 = data.groupby(["user_id","date"])

sleep1 = 0
sleep2 = 0
sleep3 = 0
delay1 = 0
delay2 = 0
delay3 = 0
for id,d in df1:
    count = 0
    cat_temp = ''
    time_temp = ''
    social = [0]*3
    game = [0]*3
    edu = [0]*3
    period = ''
    delay = 0
    print(id)
#     print("===========================================")
    for index,row in d.iterrows():
#         print(len(d))
#         print(count)
        count += 1
        if cat_temp == '':
            cat_temp = row['cat']
            time_temp = row['time']
            if (row['time'] >= datetime.time(7) and row['time'] < datetime.time(16)):
                period = 0 
            elif (row['time'] >= datetime.time(16) and row['time'] < datetime.time(22)):
                period = 1
            else:
                period = 2
            continue
#         print(row['cat'],cat_temp)
#         print(row['time'],time_temp)
        
        if (row['time'] >= datetime.time(7) and row['time'] < datetime.time(16)):
            
#             print("period: ", period)
#             print("SCHOOL")
            
            if row['cat'] == cat_temp:
                duration = datetime.datetime.combine(datetime.date(2020,1,1),row['time']) - datetime.datetime.combine(datetime.date(2020,1,1),time_temp)
                sleep_time = datetime.datetime.combine(datetime.date(2020,1,1),datetime.time(7)) - datetime.datetime.combine(datetime.date(2020,1,1),time_temp)
                school_time = duration - sleep_time
                if cat_temp == 'social':
                    if period == 2:
                        social[2] += sleep_time.total_seconds()/3600
                        social[0] += school_time.total_seconds()/3600
                    else:
                        social[0] += duration.total_seconds()/3600
                elif cat_temp == 'game':
                    if period == 2:
                        game[2] += sleep_time.total_seconds()/3600
                        game[0] += school_time.total_seconds()/3600
                    else:
                        game[0] += duration.total_seconds()/3600
                elif cat_temp == 'edu':
                    if period == 2:
                        edu[2] += sleep_time.total_seconds()/3600
                        edu[0] += school_time.total_seconds()/3600
                    else:
                        edu[0] += duration.total_seconds()/3600
                
            cat_temp = row['cat']
            time_temp = row['time']
            period = 0
                
                
            
        elif (row['time'] >= datetime.time(16) and row['time'] < datetime.time(22)):
            
#             print("period: ", period)
#             print("EVENING")
            
            if row['cat'] == cat_temp :
                duration = datetime.datetime.combine(datetime.date(2020,1,1),row['time']) - datetime.datetime.combine(datetime.date(2020,1,1),time_temp)
                school_time  = datetime.datetime.combine(datetime.date(2020,1,1),datetime.time(16)) - datetime.datetime.combine(datetime.date(2020,1,1),time_temp)
                sleep_time = datetime.datetime.combine(datetime.date(2020,1,1),datetime.time(7)) - datetime.datetime.combine(datetime.date(2020,1,1),time_temp)
                eve_time = duration - school_time
                if cat_temp == 'social':
                    if period == 0:
                        social[0] += school_time.total_seconds()/3600
                        social[1] += eve_time.total_seconds()/3600
                    elif period == 2:
                        social[2] += sleep_time.total_seconds()/3600
                        social[0] += 9
                        social[1] += eve_time.total_seconds()/3600
                    else:
                        social[1] += duration.total_seconds()/3600
                elif cat_temp == 'game':
                    if period == 0:
                        game[0] += school_time.total_seconds()/3600
                        game[1] += eve_time.total_seconds()/3600
                    elif period == 2:
                        game[2] += sleep_time.total_seconds()/3600
                        game[0] += 9
                        game[1] += eve_time.total_seconds()/3600
                    else:
                        game[1] += duration.total_seconds()/3600
                elif cat_temp == 'edu':
                    if period == 0:
                        edu[0] += school_time.total_seconds()/3600
                        edu[1] += eve_time.total_seconds()/3600
                    elif period == 2:
                        edu[2] += sleep_time.total_seconds()/3600
                        edu[0] += 9
                        edu[1] += eve_time.total_seconds()/3600
                    else:
                        edu[1] += duration.total_seconds()/3600
                
            if count == len(d):
                _delay = datetime.datetime.combine(datetime.date(2020,1,1),row['time']) - datetime.datetime.combine(datetime.date(2020,1,1),datetime.time(22))
                delay = math.ceil(_delay.total_seconds()/3600)
#                 print("Eve",delay, row['time'])
                if delay < -3:
                    delay = -3
#                 print(delay)
            cat_temp = row['cat']
            time_temp = row['time']
            period = 1    
                
            
        else:
            
#             print("period: ", period)
#             print("SlEEP")
           
            if row['cat'] == cat_temp:
                duration = datetime.datetime.combine(datetime.date(2020,1,1),row['time']) - datetime.datetime.combine(datetime.date(2020,1,1),time_temp)
                eve_time  = datetime.datetime.combine(datetime.date(2020,1,1),datetime.time(22)) - datetime.datetime.combine(datetime.date(2020,1,1),time_temp)
                school_time  = datetime.datetime.combine(datetime.date(2020,1,1),datetime.time(16)) - datetime.datetime.combine(datetime.date(2020,1,1),time_temp)
                sleep_time = duration - eve_time
                if cat_temp == 'social':
                    if period == 1:
                        social[1] += eve_time.total_seconds()/3600
                        social[2] += sleep_time.total_seconds()/3600
                    elif period == 0:
                        social[0] += school_time.total_seconds()/3600
                        social[1] += 6
                        social[2] += sleep_time.total_seconds()/3600
                    else:
                        social[2] += duration.total_seconds()/3600
                elif cat_temp == 'game':
                    if period == 1:
                        game[1] += eve_time.total_seconds()/3600
                        game[2] += sleep_time.total_seconds()/3600
                    elif period == 0:
                        game[0] += school_time.total_seconds()/3600
                        game[1] += 6
                        game[2] += sleep_time.total_seconds()/3600
                    else:
                        game[2] += duration.total_seconds()/3600
                elif cat_temp == 'edu':
                    if period == 1:
                        edu[1] += eve_time.total_seconds()/3600
                        edu[2] += sleep_time.total_seconds()/3600
                    elif period == 0:
                        edu[0] += school_time.total_seconds()/3600
                        edu[1] += 6
                        edu[2] += sleep_time.total_seconds()/3600
                    else:
                        edu[2] += duration.total_seconds()/3600
            if count == len(d):
                if row['time'] > datetime.time(22):
                    _delay = datetime.datetime.combine(datetime.date(2020,1,1),row['time']) - datetime.datetime.combine(datetime.date(2020,1,1),datetime.time(22))
                    delay = math.ceil(_delay.total_seconds()/3600)
#                     print("Sleep",delay, row['time'])
                    if delay > 3:
                        delay = 3
#                     print(delay)
                else:
                    _delay = datetime.datetime.combine(datetime.date(2020,1,1),row['time']) - datetime.datetime.combine(datetime.date(2020,1,1),datetime.time(0))
                    delay = math.ceil(_delay.total_seconds()/3600)+ 2
#                     print("Sleep",delay, row['time'])
                    if delay > 4:
                        delay = 4
#                     print(delay)
            cat_temp = row['cat']
            time_temp = row['time']
            period = 2  
#         print(social,edu,game)        
            
    social = [math.ceil(i) for i in social]
    game = [math.ceil(i) for i in game]  
    edu = [math.ceil(i) for i in edu]  
    
    
    
    df_out.loc[df_out.index.max()+1] = social + edu + game + [9-sleep1, delay1, 9-sleep2, delay2, 9-sleep3, delay3]
    sleep1 = sleep2
    sleep2 = sleep3
    sleep3 = social[2] + edu[2] + game[2]
    delay1 = delay2
    delay2 = delay3
    delay3 = delay
    


(0, datetime.date(2016, 4, 20))
(0, datetime.date(2016, 4, 21))
(0, datetime.date(2016, 4, 22))
(0, datetime.date(2016, 4, 23))
(0, datetime.date(2016, 4, 24))
(0, datetime.date(2016, 4, 25))
(0, datetime.date(2016, 4, 26))
(1, datetime.date(2016, 4, 20))
(1, datetime.date(2016, 4, 21))
(1, datetime.date(2016, 4, 22))
(1, datetime.date(2016, 4, 23))
(1, datetime.date(2016, 4, 24))
(1, datetime.date(2016, 4, 25))
(1, datetime.date(2016, 4, 26))
(2, datetime.date(2016, 4, 21))
(2, datetime.date(2016, 4, 22))
(2, datetime.date(2016, 4, 23))
(2, datetime.date(2016, 4, 24))
(2, datetime.date(2016, 4, 25))
(2, datetime.date(2016, 4, 26))
(3, datetime.date(2016, 4, 20))
(3, datetime.date(2016, 4, 21))
(3, datetime.date(2016, 4, 22))
(3, datetime.date(2016, 4, 23))
(3, datetime.date(2016, 4, 24))
(3, datetime.date(2016, 4, 25))
(3, datetime.date(2016, 4, 26))
(4, datetime.date(2016, 4, 20))
(4, datetime.date(2016, 4, 21))
(4, datetime.date(2016, 4, 22))
(4, datetime.date(2016, 4, 23))
(4, date

(44, datetime.date(2016, 4, 22))
(44, datetime.date(2016, 4, 23))
(44, datetime.date(2016, 4, 24))
(44, datetime.date(2016, 4, 25))
(44, datetime.date(2016, 4, 26))
(46, datetime.date(2016, 4, 20))
(46, datetime.date(2016, 4, 21))
(46, datetime.date(2016, 4, 22))
(46, datetime.date(2016, 4, 25))
(46, datetime.date(2016, 4, 26))
(47, datetime.date(2016, 4, 20))
(47, datetime.date(2016, 4, 21))
(47, datetime.date(2016, 4, 22))
(47, datetime.date(2016, 4, 23))
(47, datetime.date(2016, 4, 24))
(47, datetime.date(2016, 4, 25))
(47, datetime.date(2016, 4, 26))
(48, datetime.date(2016, 4, 20))
(48, datetime.date(2016, 4, 21))
(48, datetime.date(2016, 4, 22))
(48, datetime.date(2016, 4, 23))
(48, datetime.date(2016, 4, 24))
(48, datetime.date(2016, 4, 25))
(48, datetime.date(2016, 4, 26))
(49, datetime.date(2016, 4, 20))
(49, datetime.date(2016, 4, 21))
(49, datetime.date(2016, 4, 22))
(49, datetime.date(2016, 4, 23))
(49, datetime.date(2016, 4, 24))
(49, datetime.date(2016, 4, 25))
(49, datet

(92, datetime.date(2016, 4, 23))
(92, datetime.date(2016, 4, 24))
(92, datetime.date(2016, 4, 25))
(92, datetime.date(2016, 4, 26))
(94, datetime.date(2016, 4, 20))
(94, datetime.date(2016, 4, 21))
(94, datetime.date(2016, 4, 22))
(94, datetime.date(2016, 4, 23))
(94, datetime.date(2016, 4, 25))
(94, datetime.date(2016, 4, 26))
(95, datetime.date(2016, 4, 20))
(95, datetime.date(2016, 4, 21))
(95, datetime.date(2016, 4, 22))
(95, datetime.date(2016, 4, 23))
(95, datetime.date(2016, 4, 24))
(95, datetime.date(2016, 4, 25))
(95, datetime.date(2016, 4, 26))
(96, datetime.date(2016, 4, 20))
(96, datetime.date(2016, 4, 21))
(96, datetime.date(2016, 4, 22))
(96, datetime.date(2016, 4, 23))
(96, datetime.date(2016, 4, 24))
(96, datetime.date(2016, 4, 25))
(96, datetime.date(2016, 4, 26))
(98, datetime.date(2016, 4, 20))
(98, datetime.date(2016, 4, 21))
(98, datetime.date(2016, 4, 22))
(98, datetime.date(2016, 4, 23))
(98, datetime.date(2016, 4, 24))
(98, datetime.date(2016, 4, 25))
(98, datet

(141, datetime.date(2016, 4, 26))
(142, datetime.date(2016, 4, 20))
(142, datetime.date(2016, 4, 21))
(142, datetime.date(2016, 4, 22))
(142, datetime.date(2016, 4, 23))
(142, datetime.date(2016, 4, 24))
(142, datetime.date(2016, 4, 25))
(142, datetime.date(2016, 4, 26))
(143, datetime.date(2016, 4, 21))
(143, datetime.date(2016, 4, 22))
(143, datetime.date(2016, 4, 25))
(144, datetime.date(2016, 4, 20))
(144, datetime.date(2016, 4, 21))
(144, datetime.date(2016, 4, 22))
(144, datetime.date(2016, 4, 23))
(144, datetime.date(2016, 4, 24))
(144, datetime.date(2016, 4, 26))
(145, datetime.date(2016, 4, 20))
(145, datetime.date(2016, 4, 21))
(145, datetime.date(2016, 4, 22))
(145, datetime.date(2016, 4, 23))
(145, datetime.date(2016, 4, 24))
(145, datetime.date(2016, 4, 25))
(145, datetime.date(2016, 4, 26))
(147, datetime.date(2016, 4, 20))
(147, datetime.date(2016, 4, 21))
(147, datetime.date(2016, 4, 22))
(147, datetime.date(2016, 4, 23))
(147, datetime.date(2016, 4, 24))
(147, datetime

(190, datetime.date(2016, 4, 22))
(190, datetime.date(2016, 4, 24))
(190, datetime.date(2016, 4, 25))
(190, datetime.date(2016, 4, 26))
(191, datetime.date(2016, 4, 20))
(191, datetime.date(2016, 4, 25))
(191, datetime.date(2016, 4, 26))
(192, datetime.date(2016, 4, 20))
(192, datetime.date(2016, 4, 21))
(192, datetime.date(2016, 4, 22))
(192, datetime.date(2016, 4, 23))
(192, datetime.date(2016, 4, 24))
(192, datetime.date(2016, 4, 25))
(192, datetime.date(2016, 4, 26))
(193, datetime.date(2016, 4, 20))
(193, datetime.date(2016, 4, 21))
(193, datetime.date(2016, 4, 22))
(193, datetime.date(2016, 4, 23))
(193, datetime.date(2016, 4, 24))
(193, datetime.date(2016, 4, 25))
(194, datetime.date(2016, 4, 21))
(194, datetime.date(2016, 4, 22))
(194, datetime.date(2016, 4, 23))
(194, datetime.date(2016, 4, 24))
(194, datetime.date(2016, 4, 25))
(194, datetime.date(2016, 4, 26))
(196, datetime.date(2016, 4, 20))
(196, datetime.date(2016, 4, 21))
(196, datetime.date(2016, 4, 22))
(196, datetime

...

(782, datetime.date(2016, 4, 21))
(782, datetime.date(2016, 4, 22))
(782, datetime.date(2016, 4, 23))
(782, datetime.date(2016, 4, 24))
(782, datetime.date(2016, 4, 25))
(782, datetime.date(2016, 4, 26))
(783, datetime.date(2016, 4, 20))
(783, datetime.date(2016, 4, 21))
(783, datetime.date(2016, 4, 22))
(783, datetime.date(2016, 4, 23))
(783, datetime.date(2016, 4, 24))
(783, datetime.date(2016, 4, 25))
(783, datetime.date(2016, 4, 26))
(784, datetime.date(2016, 4, 20))
(784, datetime.date(2016, 4, 21))
(784, datetime.date(2016, 4, 22))
(784, datetime.date(2016, 4, 24))
(784, datetime.date(2016, 4, 25))
(784, datetime.date(2016, 4, 26))
(785, datetime.date(2016, 4, 20))
(785, datetime.date(2016, 4, 21))
(785, datetime.date(2016, 4, 22))
(785, datetime.date(2016, 4, 24))
(785, datetime.date(2016, 4, 25))
(785, datetime.date(2016, 4, 26))
(786, datetime.date(2016, 4, 20))
(786, datetime.date(2016, 4, 21))
(786, datetime.date(2016, 4, 22))
(786, datetime.date(2016, 4, 24))
(786, datetime

(826, datetime.date(2016, 4, 20))
(826, datetime.date(2016, 4, 21))
(826, datetime.date(2016, 4, 22))
(826, datetime.date(2016, 4, 23))
(826, datetime.date(2016, 4, 25))
(826, datetime.date(2016, 4, 26))
(827, datetime.date(2016, 4, 20))
(827, datetime.date(2016, 4, 21))
(827, datetime.date(2016, 4, 22))
(828, datetime.date(2016, 4, 20))
(828, datetime.date(2016, 4, 22))
(828, datetime.date(2016, 4, 23))
(828, datetime.date(2016, 4, 24))
(828, datetime.date(2016, 4, 25))
(828, datetime.date(2016, 4, 26))
(829, datetime.date(2016, 4, 20))
(829, datetime.date(2016, 4, 21))
(829, datetime.date(2016, 4, 22))
(829, datetime.date(2016, 4, 23))
(829, datetime.date(2016, 4, 26))
(830, datetime.date(2016, 4, 20))
(830, datetime.date(2016, 4, 21))
(830, datetime.date(2016, 4, 22))
(830, datetime.date(2016, 4, 23))
(830, datetime.date(2016, 4, 24))
(830, datetime.date(2016, 4, 25))
(830, datetime.date(2016, 4, 26))
(832, datetime.date(2016, 4, 20))
(832, datetime.date(2016, 4, 21))
(832, datetime

(869, datetime.date(2016, 4, 25))
(869, datetime.date(2016, 4, 26))
(870, datetime.date(2016, 4, 20))
(870, datetime.date(2016, 4, 21))
(870, datetime.date(2016, 4, 22))
(870, datetime.date(2016, 4, 24))
(870, datetime.date(2016, 4, 25))
(870, datetime.date(2016, 4, 26))
(871, datetime.date(2016, 4, 20))
(871, datetime.date(2016, 4, 21))
(871, datetime.date(2016, 4, 22))
(871, datetime.date(2016, 4, 23))
(871, datetime.date(2016, 4, 24))
(871, datetime.date(2016, 4, 25))
(871, datetime.date(2016, 4, 26))
(872, datetime.date(2016, 4, 20))
(872, datetime.date(2016, 4, 21))
(872, datetime.date(2016, 4, 22))
(872, datetime.date(2016, 4, 23))
(872, datetime.date(2016, 4, 24))
(872, datetime.date(2016, 4, 25))
(872, datetime.date(2016, 4, 26))
(873, datetime.date(2016, 4, 20))
(873, datetime.date(2016, 4, 23))
(873, datetime.date(2016, 4, 24))
(874, datetime.date(2016, 4, 20))
(874, datetime.date(2016, 4, 21))
(874, datetime.date(2016, 4, 22))
(874, datetime.date(2016, 4, 24))
(874, datetime

(914, datetime.date(2016, 4, 22))
(914, datetime.date(2016, 4, 23))
(914, datetime.date(2016, 4, 24))
(914, datetime.date(2016, 4, 25))
(914, datetime.date(2016, 4, 26))
(915, datetime.date(2016, 4, 20))
(915, datetime.date(2016, 4, 21))
(915, datetime.date(2016, 4, 22))
(915, datetime.date(2016, 4, 24))
(915, datetime.date(2016, 4, 25))
(915, datetime.date(2016, 4, 26))
(916, datetime.date(2016, 4, 20))
(916, datetime.date(2016, 4, 21))
(916, datetime.date(2016, 4, 22))
(916, datetime.date(2016, 4, 23))
(916, datetime.date(2016, 4, 24))
(916, datetime.date(2016, 4, 25))
(916, datetime.date(2016, 4, 26))
(917, datetime.date(2016, 4, 20))
(917, datetime.date(2016, 4, 21))
(917, datetime.date(2016, 4, 22))
(917, datetime.date(2016, 4, 23))
(917, datetime.date(2016, 4, 24))
(917, datetime.date(2016, 4, 25))
(917, datetime.date(2016, 4, 26))
(919, datetime.date(2016, 4, 20))
(919, datetime.date(2016, 4, 21))
(919, datetime.date(2016, 4, 22))
(919, datetime.date(2016, 4, 24))
(919, datetime

(962, datetime.date(2016, 4, 20))
(962, datetime.date(2016, 4, 21))
(963, datetime.date(2016, 4, 20))
(963, datetime.date(2016, 4, 21))
(963, datetime.date(2016, 4, 22))
(963, datetime.date(2016, 4, 23))
(963, datetime.date(2016, 4, 24))
(963, datetime.date(2016, 4, 25))
(963, datetime.date(2016, 4, 26))
(964, datetime.date(2016, 4, 20))
(964, datetime.date(2016, 4, 21))
(964, datetime.date(2016, 4, 22))
(964, datetime.date(2016, 4, 23))
(964, datetime.date(2016, 4, 24))
(964, datetime.date(2016, 4, 25))
(964, datetime.date(2016, 4, 26))
(965, datetime.date(2016, 4, 20))
(965, datetime.date(2016, 4, 21))
(965, datetime.date(2016, 4, 22))
(965, datetime.date(2016, 4, 23))
(965, datetime.date(2016, 4, 24))
(965, datetime.date(2016, 4, 25))
(965, datetime.date(2016, 4, 26))
(966, datetime.date(2016, 4, 20))
(966, datetime.date(2016, 4, 21))
(966, datetime.date(2016, 4, 22))
(966, datetime.date(2016, 4, 24))
(966, datetime.date(2016, 4, 25))
(966, datetime.date(2016, 4, 26))
(967, datetime

In [11]:
df_out

,Sc_Social,Ev_Social,Sl_Social,Sc_Edu,Ev_Edu,Sl_Edu,Sc_Game,Ev_Game,Sl_Game,Day1_Hs,Day1_D,Day2_Hs,Day2_D,Day3_Hs,Day3_D
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,0,1,1,0,5,4,0,9,0,9,0,9,0
2,2,1,0,1,1,0,1,1,0,9,0,9,0,9,-1
3,2,1,0,1,1,0,4,1,5,9,0,9,-1,9,0
4,2,1,0,1,1,0,1,1,0,9,-1,9,0,4,0
5,2,0,0,1,0,0,1,0,0,9,0,4,0,9,-3
6,2,1,0,1,0,0,1,2,0,4,0,9,-3,9,0
7,3,0,0,1,0,0,2,0,0,9,-3,9,0,9,-2
8,0,1,0,0,1,0,8,2,0,9,0,9,-2,9,0
9,6,1,0,1,2,0,3,2,0,9,-2,9,0,9,0


In [12]:
df_out = df_out.drop([0,1,2,3])
df_out = df_out.reset_index(drop=True)
df_out

,Sc_Social,Ev_Social,Sl_Social,Sc_Edu,Ev_Edu,Sl_Edu,Sc_Game,Ev_Game,Sl_Game,Day1_Hs,Day1_D,Day2_Hs,Day2_D,Day3_Hs,Day3_D
0,2,1,0,1,1,0,1,1,0,9,-1,9,0,4,0
1,2,0,0,1,0,0,1,0,0,9,0,4,0,9,-3
2,2,1,0,1,0,0,1,2,0,4,0,9,-3,9,0
3,3,0,0,1,0,0,2,0,0,9,-3,9,0,9,-2
4,0,1,0,0,1,0,8,2,0,9,0,9,-2,9,0
5,6,1,0,1,2,0,3,2,0,9,-2,9,0,9,0
6,3,3,0,1,0,0,5,0,0,9,0,9,0,9,0
7,1,0,0,1,1,0,0,1,0,9,0,9,0,9,-3
8,2,1,0,0,0,0,3,2,0,9,0,9,-3,9,0
9,3,3,0,0,0,0,1,1,0,9,-3,9,0,9,-3


In [13]:
df_out.to_csv('out.csv', encoding='utf-8', index=False)